In [1]:
import torch
from torch import nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, models, transforms
from torchvision.transforms import ToTensor
from torchvision.datasets import ImageFolder


In [2]:
train_transforms = transforms.Compose([
    transforms.Resize((224, 224)),  # 调整图片大小为 224x224
    transforms.Grayscale(num_output_channels=3),  # 将灰度图像转换为 3 通道的图像
    transforms.RandomHorizontalFlip(),  # 随机水平翻转
    transforms.ToTensor(),  # 转换为 tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # 使用ImageNet均值和标准差
])

test_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.Grayscale(num_output_channels=3),  # 同样转换为 3 通道
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

train_dir = "./data/train"
test_dir = "./data/test"
train_data = ImageFolder(train_dir, transform=train_transforms)
test_data = ImageFolder(test_dir, transform=test_transforms)
 
train_loader = DataLoader(train_data, batch_size=64, shuffle=True, pin_memory=True)
test_loader = DataLoader(test_data, batch_size=64, shuffle=True, pin_memory=True)

                        

In [3]:
print(len(train_data),len(test_data))

28710 7179


In [4]:

for X, y in test_loader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 3, 224, 224])
Shape of y: torch.Size([64]) torch.int64


In [5]:
# 加载预训练的 ResNet-50 模型
# 1. 初始化 ResNet50 模型结构（没有加载预训练权重）
model = models.resnet50(weights=None)  # 初始化模型结构，不加载预训练权重

# 2. 加载本地保存的模型权重
model_path = '/hy-tmp/model/resnet50.pth'
# 使用 weights_only=True 来确保只加载模型的权重
model.load_state_dict(torch.load(model_path, weights_only=True))


# 修改最后的全连接层，以适应 FER-2013 数据集的情绪分类任务
# FER-2013 数据集有 7 个类别
model.fc = torch.nn.Linear(model.fc.in_features, 7)

# 将模型移动到 GPU（如果可用）
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [6]:


# 定义损失函数
criterion = torch.nn.CrossEntropyLoss()

# 定义优化器
optimizer = optim.Adam(model.parameters(), lr=1e-4)


In [7]:
from tqdm import tqdm  # 导入tqdm库

num_epochs = 5
# 假设你已经有一个模型和优化器，并且训练和测试的数据加载器已经准备好
for epoch in range(num_epochs):  # 循环所有的epoch
    model.train()  # 设置模型为训练模式
    running_loss = 0.0
    correct = 0
    total = 0
    
    # 使用tqdm包装train_loader，显示进度条
    # train_loader是你用于训练的DataLoader
    for i, (inputs, labels) in enumerate(tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}", unit="batch")):
        # 清空梯度
        optimizer.zero_grad()
        
        # 将输入数据和标签放到设备上
        inputs, labels = inputs.to(device), labels.to(device)
        
        # 前向传播
        outputs = model(inputs)
        
        # 计算损失
        loss = criterion(outputs, labels)
        
        # 反向传播
        loss.backward()
        
        # 更新模型参数
        optimizer.step()
        
        # 累加损失
        running_loss += loss.item()
        
        # 获取预测的标签
        _, predicted = torch.max(outputs, 1)
        
        # 累加正确预测的样本数
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    # 输出当前epoch的平均损失和准确率
    avg_loss = running_loss / len(train_loader)
    accuracy = (correct / total) * 100
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}, Accuracy: {accuracy:.2f}%")


Epoch 1/5: 100%|██████████| 449/449 [06:25<00:00,  1.16batch/s]


Epoch [1/5], Loss: 1.0990, Accuracy: 58.54%


Epoch 2/5: 100%|██████████| 449/449 [06:08<00:00,  1.22batch/s]


Epoch [2/5], Loss: 0.8529, Accuracy: 68.20%


Epoch 3/5: 100%|██████████| 449/449 [07:06<00:00,  1.05batch/s]


Epoch [3/5], Loss: 0.7162, Accuracy: 73.82%


Epoch 4/5: 100%|██████████| 449/449 [07:53<00:00,  1.06s/batch]


Epoch [4/5], Loss: 0.5979, Accuracy: 78.37%


Epoch 5/5: 100%|██████████| 449/449 [07:08<00:00,  1.05batch/s]

Epoch [5/5], Loss: 0.4647, Accuracy: 83.47%


In [8]:
# 保存模型
torch.save(model.state_dict(), './finetuned_model/FER1.pth')
